

```
NLP Assignment 5
Submitted by:
  Aleezeh Usman
  18I-0529
  i180529@nu.edu.pk
```



# **Fake News Detection** 

import required libraries and get data, then organize it in usable format

In [ ]:
!pip install spacy
import csv
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#declaring important variables
realcorpus = list()             #entire list of words in the real news corpus
fakecorpus = list()             #entire list of words in the fake news corpus
fakevocab = dict()              #this will store all unique words and the count of each word
realvocab = dict()
stopwords = list()
realtestcorpus = list()
faketestcorpus = list()
punctuation = ['\n','۔','،','!','؟','?','!','.','"','’','‘','/','\\','%','٪','’’','‘‘']
       

In [ ]:
with open("drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/stopwords-ur.txt", 'r') as f:
  reader = csv.reader(f)
  stopwords = list(reader)
f.close()

print(len(stopwords))

517


In [ ]:
#to read all the files in a folder get all the file names first
realfileslist = os.listdir("drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Train/Real")
fakefileslist = os.listdir("drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Train/Fake")
realtestfiles = os.listdir("drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Test/Real")
faketestfiles = os.listdir("drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Test/Fake")
#print(len(realfileslist))
#print(len(fakefileslist))
#print(len(realtestfiles))
#print(len(faketestfiles))

In [ ]:
#now read contents of each file and store in a list 
for each in realfileslist:
  path = 'drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Train/Real/' + each
  with open(path, 'r') as f:
    realfilecontent = f.read()
  f.close()
  realcorpus.append(realfilecontent) 

for each in fakefileslist:
  path = 'drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Train/Fake/' + each
  with open(path, 'r') as f:
    fakefilecontent = f.read()
  f.close()
  fakecorpus.append(fakefilecontent) 

for each in realtestfiles:
  path = 'drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Test/Real/' + each
  with open(path, 'r') as f:
    realtestcontent = f.read()
  f.close()
  realtestcorpus.append(realtestcontent)

for each in faketestfiles:
  path = 'drive/My Drive/Classroom/Natural Language Processing (Spring 2021)/Notebooks/assignment_5/data/Test/Real/' + each
  with open(path, 'r') as f:
    faketestcontent = f.read()
  f.close()
  faketestcorpus.append(faketestcontent)

print(len(realtestcorpus))
print(len(faketestcorpus))

150
112


In [ ]:
#this will generate count the occurences of each word to build a vocabulary that can be used later on
def UnigramModel(training_set, total_words):
  probabilities = dict()
  i = 0

  while(i < total_words):                               # in a single loop simply count occurences of each word
    if str(training_set[i]) not in punctuation:
      if str(training_set[i]) in probabilities:
        probabilities[str(training_set[i])] += 1
      else:
        probabilities[str(training_set[i])] = 1
    i += 1

  return probabilities

In [ ]:
import spacy as sp

#convert entire real corpus to a realvocab
tempstring = ""
for each in realcorpus:
  tempstring += each
nlp = sp.blank('ur')
tempvocab = nlp(tempstring)
realvocab = dict()
realvocab = UnigramModel(tempvocab, len(tempvocab))


#convert entire fake corpus to a fake vocab
tempstring = ""
for each in fakecorpus:
  tempstring += each
nlp = sp.blank('ur')
tempvocab = nlp(tempstring)
fakevocab = dict()
fakevocab = UnigramModel(tempvocab, len(tempvocab))


# **IMPLEMENTATION**
Now we have contents of each document stored separately in realcorpus and fakecorpus, and contents of all the documents of the real files stored together as a single dictionary with their counts in realvocab and vice versa for fakevocab. Prior is a dictionary with information about the two classes but it is 0 right now, and condprob dict is declared but empty, , now we will begin the work. 

In [ ]:
import spacy as sp

#function for boolean NB implementation i.e to remove duplicate words from each document
def ConvertToBooleanCorpus(corpus):
  toreturncorpus = list()
  nlp = sp.blank('ur')

  for eachdoc in corpus:
    tempstring = ""
    tempstore = nlp(eachdoc)
    for word in tempstore:
      if str(word) not in tempstring:
        tempstring += str(word) 
    toreturncorpus.append(tempstring)

  return toreturncorpus

In [ ]:
#Functions required in the Train Mulinomial NB function
import spacy as sp

def GetVocabulary(fakecorpus, realcorpus):
  list_vocab = list()
  nlp = sp.blank('ur')
  
  #store all words from real corpus in a list
  for each in realcorpus:                  #get words in a sentence 
    tempvocab = nlp(each)
    for word in tempvocab:                  #now append each word individually in the list storing words
      if str(word) not in punctuation:
       list_vocab.append(str(word))
  #store all words from fake corpus in a list
  for each in fakecorpus:
    tempvocab = nlp(each)
    for word in tempvocab:
      if str(word) not in punctuation:
        list_vocab.append(str(word))

  return list_vocab

def CountWordsInAllTextsofClass(vocab):
  count = 0
  for each in vocab:
    count += vocab[each]
  return count

def CountTokensOfWords(doc_c, w):
  count = 0
  if w in doc_c:
    count = doc_c[w]
  return count

def ExtractWordsfromText(textdoc):
  toreturn = list()
  nlp = sp.blank('ur')
  tempvocab = nlp(textdoc)
  for word in tempvocab:    
    if str(word) not in punctuation:
      toreturn.append(str(word))

  return toreturn

In [ ]:
def InternalLoopforTrainMultinomailNB(V, vocab):
  condprob = dict()
  Nw = CountWordsInAllTextsofClass(vocab) #number of total words in all the text files in the corpus of that class
  for eachword in vocab:
    Ni = CountTokensOfWords(vocab, eachword)
    cp = (Ni+1)/(Nw + len(V))
    condprob[eachword] = cp               #each word will store the corresponding probability of having that word in the given class
  
  toReturnList =  condprob
  return toReturnList

In [ ]:
def TrainMultinomialNB(realvocab, fakevocab, realcorpus, fakecorpus, stopwords, removestopwords):
  N = len(realcorpus) + len(fakecorpus)               #total number of text documents in both classes
  V = GetVocabulary(fakecorpus, realcorpus)           #Complete Vocabulary with duplicates and stopwords too
  if removestopwords == 1:                            #remove stopwords or not
    for each in stopwords:
      if each[0] in V:
        try:
          while True:
            V.remove(each[0])
        except ValueError:
          pass
  print(len(V))

  #Calculate everything for real news
  Nc = len(realcorpus)                                #number of text documents of the real news corpus
  prior['R'] = Nc/N                                   #calculate prior 
  GetInfo = InternalLoopforTrainMultinomailNB(V,realvocab)  
  condprob['R'] = GetInfo.copy()                      #store entire dict containing cond prob of each word being in real class

  #Calculate everything for fake news
  Nc = len(fakecorpus)                                #number of text documents of the real news corpus
  prior['F'] = Nc/N
  GetInfo = InternalLoopforTrainMultinomailNB(V,fakevocab)
  condprob['F'] = GetInfo.copy()                      #store entire dict containing cond prob of each word being in fake class


In [ ]:
from math import log

def NaiveBayesModel(prior, condprob, testcorpus, stopwords, removestopwords):
  W = ExtractWordsfromText(testcorpus)
  if removestopwords == 1:
    for each in stopwords:                        #remove stopwords if required
      if each[0] in W:
        try:
          while True:
            W.remove(each[0])
        except ValueError:
          pass
  score = dict()

  score['R'] = log(prior['R'])                  #calculate score of the doc as part of real(R) class
  for w in W:
    if w in condprob['R']:
      score['R'] += log(condprob['R'][w])
  
  score['F'] = log(prior['F'])                  #calculate score of the doc as part of fake(F) class
  for w in W:
    if w in condprob['F']:
      score['F'] += log(condprob['F'][w])
  
  if score['F'] > score['R']:                   #return the correct label based on max score
    return 'F'
  else:
    return 'R'

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def EvaluateCode(realtestcorpus,faketestcorpus, prior, condprob, stopwords, removestopwords):
  predicted_labels = list()                             #store labels that we should be getting
  actual_labels = list()                                #store labels that we actually got from our algorithm

  for doc in realtestcorpus:                            #get label for each document in our data set separately
    decision = NaiveBayesModel(prior, condprob, doc, stopwords, removestopwords)
    predicted_labels.append(decision)
    actual_labels.append('R')                           #this loop is for real news documents so predicted label is R

  for doc in faketestcorpus:
    decision = NaiveBayesModel(prior, condprob, doc, stopwords, removestopwords)
    predicted_labels.append(decision)
    actual_labels.append('F')                           #this loop is for fake news documents so predicted label is F
    
  #evaluate and store the scores of evaluation measures
  accuracy = accuracy_score(actual_labels, predicted_labels)
  precision = precision_score(actual_labels, predicted_labels, average = 'macro')
  recall = recall_score(actual_labels, predicted_labels, average='macro')
  f1score = f1_score(actual_labels, predicted_labels, average='macro')
  #display evaluation results
  print("Accuracy: ", end = "")
  print(accuracy)
  print("Precision: ", end = "")
  print(precision)
  print("Recall: ", end = "")
  print(recall)
  print("F1 Measure: ", end = "")
  print(accuracy)

# **FINAL IMPLEMENTATION - WITHOUT STOP WORDS REMOVAL**

In [ ]:
prior = {"R": 0, "F":0}         #this is our basic distinct classes and will store the prior value
condprob = dict()               # we will calculate the cond probability of each word later
removestopwords = 0
TrainMultinomialNB(realvocab,fakevocab,realcorpus,fakecorpus,stopwords,removestopwords)
print(prior)
EvaluateCode(realtestcorpus,faketestcorpus,prior,condprob,stopwords,removestopwords)

215382
{'R': 0.54858934169279, 'F': 0.45141065830721006}
Accuracy: 0.48091603053435117
Precision: 0.49124999999999996
Recall: 0.49124999999999996
F1 Measure: 0.48091603053435117


# **FINAL IMPLEMENTATION - WITHOUT STOPWORDS**

In [ ]:
#remove stopwords from the real and fake news vocabularies
for each in stopwords:
  if each[0] in realvocab:
    realvocab.pop(each[0])
  if each[0] in fakevocab:
    fakevocab.pop(each[0])
#print(len(realvocab))
#print(len(fakevocab))

prior = {"R": 0, "F":0}         #this is our basic distinct classes and will store the prior value
condprob = dict()               # we will calculate the cond probability of each word later
removestopwords = 1
TrainMultinomialNB(realvocab,fakevocab,realcorpus,fakecorpus,stopwords,removestopwords)
print(prior)
EvaluateCode(realtestcorpus,faketestcorpus,prior,condprob,stopwords,removestopwords)

167168
{'R': 0.54858934169279, 'F': 0.45141065830721006}
Accuracy: 0.46564885496183206
Precision: 0.49043606206527557
Recall: 0.4914880952380952
F1 Measure: 0.46564885496183206


# **FINAL IMPLEMENTATION - BOOLEAN NB**

In [ ]:
booleanrealcorpus = list()
booleanfakecorpus = list()

booleanrealcorpus = ConvertToBooleanCorpus(realcorpus)
booleanfakecorpus = ConvertToBooleanCorpus(fakecorpus)

import spacy as sp
#convert entire boolean real corpus to a boolean realvocab
tempstring = ""
for each in booleanrealcorpus:
  tempstring += each
nlp = sp.blank('ur')
tempvocab = nlp(tempstring)
booleanrealvocab = dict()
booleanrealvocab = UnigramModel(tempvocab, len(tempvocab))
#convert entire boolean fake corpus to a boolean fake vocab
tempstring = ""
for each in booleanfakecorpus:
  tempstring += each
nlp = sp.blank('ur')
tempvocab = nlp(tempstring)
booleanfakevocab = dict()
booleanfakevocab = UnigramModel(tempvocab, len(tempvocab))

#remove stopwords from fake and real boolean vocab
for each in stopwords:
  if each[0] in booleanrealvocab:
    booleanrealvocab.pop(each[0])
  if each[0] in booleanfakevocab:
    booleanfakevocab.pop(each[0])

In [ ]:
#set up test corpuses  for boolean as well
booleanrealtestcorpus = list()
booleanfaketestcorpus = list()
booleanfaketestcorpus = ConvertToBooleanCorpus(faketestcorpus)
booleanrealtestcorpus = ConvertToBooleanCorpus(realtestcorpus)

In [ ]:
prior = {"R": 0, "F":0}         #this is our basic distinct classes and will store the prior value
condprob = dict()               # we will calculate the cond probability of each word later
removestopwords = 1
TrainMultinomialNB(booleanrealvocab,booleanfakevocab,booleanrealcorpus,booleanfakecorpus,stopwords,removestopwords)
print(prior)
EvaluateCode(booleanrealtestcorpus,booleanfaketestcorpus,prior,condprob,stopwords,removestopwords)

1884
{'R': 0.54858934169279, 'F': 0.45141065830721006}
Accuracy: 0.5534351145038168
Precision: 0.4923042770460263
Recall: 0.4969047619047619
F1 Measure: 0.5534351145038168




```
From the above results we can see that by removing duplicates in a file i.e Boolean Naive Bayes gives us better results as compared to simple Naive Bayes and also better than simply removing stop words. 
According to my results only removing stop words has very less change in the results. 
```

